# Testing Any Model

In [54]:
import os

os.environ["MKL_NUM_THREADS"]="1"
os.environ["NUMEXPR_NUM_THREADS"]="1"
os.environ["OMP_NUM_THREADS"]="1"

import sys

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

from spatial.merfish_dataset import FilteredMerfishDataset, MerfishDataset
from spatial.models.monet_ae import MonetAutoencoder2D, TrivialAutoencoder
from spatial.train import train
from spatial.predict import test

In [55]:
# equivalent to spatial

import hydra
from hydra.experimental import compose, initialize

with initialize(config_path="../config"):
    cfg_from_terminal = compose(config_name="config")
    # update the behavior to get the model of interest
    OmegaConf.update(cfg_from_terminal, "datasets.dataset.behaviors", ["Parenting"])
    output = test(cfg_from_terminal)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'predict/default': Usage of deprecated keyword in package head

Original Data (1027848, 170)
Filtered Data (86902, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /home/roko/spatial//output/lightning_logs/checkpoints/MonetAutoencoder2D exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.2307247370481491,
 'test_loss: mae_response': 0.36967429518699646,
 'test_loss: mse': 0.2517955005168915}
--------------------------------------------------------------------------------


In [56]:
trainer, l1_losses, inputs, gene_expressions, celltypes = output

In [57]:
with open('../spatial/non_response.txt', "r") as genes_file:
    features = [int(x) for x in genes_file.read().split(",")]
    response_indeces = torch.tensor(list(set(range(160)) - set(features)))
    genes_file.close()

In [58]:
excitatory_cells = (celltypes == 6).nonzero(as_tuple=True)[0]

In [59]:
import torch

loss = torch.nn.L1Loss()
loss(torch.index_select(inputs[excitatory_cells], 1, response_indeces), torch.index_select(gene_expressions[excitatory_cells], 1, response_indeces))

tensor(0.4182)

In [60]:
# equivalent to spatial

import hydra
from hydra.experimental import compose, initialize

with initialize(config_path="../config"):
    cfg_from_terminal = compose(config_name="config")
    # update the behavior to get the model of interest
    OmegaConf.update(cfg_from_terminal, "datasets.dataset.behaviors", ["Virgin Parenting"])
    output = test(cfg_from_terminal)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'predict/default': Usage of deprecated keyword in package head

Original Data (1027848, 170)
Filtered Data (109105, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /home/roko/spatial//output/lightning_logs/checkpoints/MonetAutoencoder2D exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.22909943759441376,
 'test_loss: mae_response': 0.3661682903766632,
 'test_loss: mse': 0.23969760537147522}
--------------------------------------------------------------------------------


In [61]:
trainer, l1_losses, inputs, gene_expressions, celltypes = output

In [62]:
excitatory_cells = (celltypes == 6).nonzero(as_tuple=True)[0]

In [63]:
import torch

loss = torch.nn.L1Loss()
loss(torch.index_select(inputs[excitatory_cells], 1, response_indeces), torch.index_select(gene_expressions[excitatory_cells], 1, response_indeces))

tensor(0.4185)

In [64]:
# equivalent to spatial

import hydra
from hydra.experimental import compose, initialize

with initialize(config_path="../config"):
    cfg_from_terminal = compose(config_name="config")
    # update the behavior to get the model of interest
    OmegaConf.update(cfg_from_terminal, "datasets.dataset.behaviors", ["Naive"])
    output = test(cfg_from_terminal)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'predict/default': Usage of deprecated keyword in package head

Original Data (1027848, 170)
Filtered Data (205348, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /home/roko/spatial//output/lightning_logs/checkpoints/MonetAutoencoder2D exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.19866959750652313,
 'test_loss: mae_response': 0.3151684105396271,
 'test_loss: mse': 0.18607354164123535}
--------------------------------------------------------------------------------


In [65]:
trainer, l1_losses, inputs, gene_expressions, celltypes = output

In [66]:
excitatory_cells = (celltypes == 6).nonzero(as_tuple=True)[0]

In [67]:
import torch

loss = torch.nn.L1Loss()
loss(torch.index_select(inputs[excitatory_cells], 1, response_indeces), torch.index_select(gene_expressions[excitatory_cells], 1, response_indeces))

tensor(0.3570)